In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import scipy as sp
import scipy.sparse as sps
from scipy.sparse import coo_matrix
import os
import matplotlib.pyplot as pyplot
%matplotlib inline 


## Evaluation or scoring?
eval = False

In [2]:
from RecLib.DataLoad import *
# Loading from CSV files...
UCM_age = toCoo('data_UCM_age.csv', 'user', 'age')
ICM_subclass = toCoo('data_ICM_sub_class.csv', 'item', 'subclass')
ICM_asset = toCoo('data_ICM_asset.csv', 'item', 'asset')
ICM_price = toCoo('data_ICM_price.csv', 'item', 'price')
UCM_region = toCoo('data_UCM_region.csv', 'user', 'region')
target_users = toNPArray('data_target_users_test.csv')
URM = toCoo('data_train.csv', 'user', 'item')

data_UCM_age.csv
user data, ID: min 2, max 30909, length 30317, unique 30317, repetitions 0, missig 1.91 %
age data, ID: min 1, max 10, length 30317, unique 10, repetitions 30307, missig 0.00 %

data_ICM_sub_class.csv
item data, ID: min 0, max 18494, length 18495, unique 18495, repetitions 0, missig 0.00 %
subclass data, ID: min 1, max 2010, length 18495, unique 1905, repetitions 16590, missig 5.18 %

data_ICM_asset.csv
item data, ID: min 0, max 18494, length 18490, unique 18490, repetitions 0, missig 0.02 %
asset data, ID: min 0, max 0, length 18490, unique 1, repetitions 18489, missig 0.00 %

data_ICM_price.csv
item data, ID: min 0, max 18494, length 18493, unique 18493, repetitions 0, missig 0.01 %
price data, ID: min 0, max 0, length 18493, unique 1, repetitions 18492, missig 0.00 %

data_UCM_region.csv
user data, ID: min 0, max 30910, length 26609, unique 26375, repetitions 234, missig 14.67 %
region data, ID: min 0, max 7, length 26609, unique 7, repetitions 26602, missig 0.00 %


In [3]:
URM = URM.tocsr()

In [4]:
## Split dataset (train % of .9999 gives similar performance on test set and competition set)
from Notebooks_utils.data_splitter import train_test_holdout
URM_train, URM_test = train_test_holdout(URM, train_perc = 0.8)


if not eval:
    URM_train = URM
else:
    from Base.Evaluation.Evaluator import EvaluatorHoldout
    evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [5]:
## SLIM BPR Recommender - MAP 0.41 on test set
## set True if you want to train and compile SLIM BPR Recommender
if False:
    from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython 
    slim_rec = SLIM_BPR_Cython(URM_train, recompile_cython=False, verbose = False)
    MAP_LIST = []
    epochsList = [300]
    batchSize = [50]
    tklist = [10]
    lrs = [1e-3]

    for epochsN in epochsList:
        for bs in batchSize:
            for tk in tklist:
                for lr in lrs:
                    do_not_display_hystory = slim_rec.fit(epochs=epochsN, batch_size=bs, sgd_mode='adagrad', learning_rate=lr, topK = tk)
                    if eval:
                        dict_scores = (evaluator_validation.evaluateRecommender(slim_rec))[0][10]
                        MAP_LIST.append(('epoch, batch, topK, lr :' + str(epochsN) + ' ' + str(bs)+ ' ' + str(tk) + ' ' + str(lr), dict_scores['MAP']))
    if eval:
        MAP_LIST

In [6]:

if False:
    from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender 
    itemKNNCBF = ItemKNNCBFRecommender(URM_train, ICM_all)
    MAP_LIST = []
    tklist = [3]
    shrinklist = [10]

    for tk in tklist:
        for sr in shrinklist:
            itemKNNCBF.fit(shrink=sr, topK = tk, similarity = 'cosine')
            if eval:
                dict_scores = (evaluator_validation.evaluateRecommender(itemKNNCBF))[0][10]
                MAP_LIST.append(('topK, shrink :' + str(tk) + ' ' + str(sr), dict_scores['MAP']))
    if eval:
        print(MAP_LIST)



In [7]:
## ItemCFKNNRecommender - MAP 0.47 on test set
# If you want to compile and train KNN set to True
if False:
    from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
    itemKNN = ItemKNNCFRecommender(URM_train)
    itemKNN.fit(shrink=50, topK=3)

    if eval:
        results, _ = evaluator_validation.evaluateRecommender(itemKNN)
        print('MAP: ' + str(results[10]["MAP"]))

In [8]:
## HYBRID: KNN + SLIM BPR
# Set to True if you want to compile and train this hybrid rec
from RecLib.HybridRecommender import *
if True: 
    hybridrecommender = HybridRecommender(URM_train)
    hybridrecommender.fit()
    
    if eval:
        results, _ = evaluator_validation.evaluateRecommender(hybridrecommender)
        print('MAP: ' + str(results[10]["MAP"]))

HybridRecommender: URM Detected 3656 (11.83 %) cold users.
HybridRecommender: URM Detected 3218 (17.40 %) cold items.
TopPopRecommender: URM Detected 3656 (11.83 %) cold users.
TopPopRecommender: URM Detected 3218 (17.40 %) cold items.
ItemKNNCFRecommender: URM Detected 3656 (11.83 %) cold users.
ItemKNNCFRecommender: URM Detected 3218 (17.40 %) cold items.
Similarity column 18495 ( 100 % ), 7400.67 column/sec, elapsed time 0.04 min


In [9]:
if eval:
    for itr in range(300):
        to_compute_mask = np.ediff1d(URM_train.tocsr().indptr) == itr
        to_ignore_mask = np.invert(to_compute_mask)
        to_ignore = np.arange(URM_train.shape[0])[to_ignore_mask]
        if len(to_ignore) != URM_train.shape[0]:
            evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users = to_ignore)
            results, _ = evaluator_validation.evaluateRecommender(hybridrecommender)
            print('MAP at ' + str(itr) +' interactions: ' + str(results[10]["MAP"]))

In [10]:
## TEST & VISUALIZE DIFFERENT RECSYS PERFORMANCE BASED ON N° OF INTERACTIONS HELD BY USERS
# Please, fill empty list with pretrained recommenders you want to compare

if eval:
    recommendersToCompare = [itemKNN, hybridrecommender]
    from RecLib.PerUserGroupTesting import *
    compare(URM_train, URM_test, recommendersToCompare)

In [11]:
## Predict
## SET WHICH RECOMMENDER YOU WANNA USE
final_Rec = hybridrecommender
if not eval:
    output = []
    for user_id in target_users:
        output.append((user_id, final_Rec.recommend(user_id, cutoff=10)))

In [12]:
#Writedown results
if not eval:
    with open('submission.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["user_id", "item_list"])
        for row in output:
          ranking = ''
          for val in row[1]:
            ranking = ranking + str(val) + ' '
          writer.writerow([row[0], ranking[:-1]])

(0, [3569, 6530, 6404, 6289, 5494, 10227, 5113, 3601, 3509, 5700])
(1, [7259, 8470, 16148, 7671, 3604, 6280, 8429, 16460, 345, 5034])
(2, [4657, 4658, 11734, 735, 739, 737, 13837, 11733, 5732, 17550])
(4, [4634, 4728, 4637, 5043, 4638, 2835, 15898, 1943, 7494, 5133])
(5, [6798, 4013, 9641, 12196, 12195, 6794, 4748, 6796, 4747, 5297])
(6, [15856, 4732, 10359, 14912, 5570, 3456, 7550, 11715, 14958, 12810])
(7, [4071, 9176, 9014, 10469, 6780, 6798, 13651, 7174, 4220, 4977])
(8, [5192, 4526, 5189, 4527, 5190, 5206, 6646, 5193, 4839, 5176])
(9, [9303, 4082, 4083, 5297, 4432, 5294, 9305, 5301, 7602, 8203])
(10, [3901, 16097, 6288, 67, 8529, 15841, 4325, 13416, 3900, 548])
(11, [18360, 2730, 4767, 9196, 17987, 4790, 4784, 4799, 4528, 5202])
(12, [4705, 3529, 3528, 3534, 4722, 4721, 3536, 6986, 9476, 9477])
(13, [4785, 1023, 5144, 5133, 4784, 5118, 4482, 5119, 2788, 5505])
(14, [7885, 5725, 6200, 6201, 4702, 7887, 9302, 4703, 9263, 7888])
(15, [18301, 18299, 12194, 17661, 6135, 3991, 10232, 39

(1572, [3572, 3595, 3576, 3594, 9208, 8404, 7592, 9475, 8982, 9205])
(1573, [9203, 9204, 9202, 9206, 15398, 9201, 4250, 4362, 9224, 4634])
(1574, [16158, 15229, 12330, 12328, 12334, 12333, 12332, 12331, 12336, 12329])
(1575, [10055, 8257, 7607, 10227, 5143, 1023, 6510, 7609, 8666, 5142])
(1576, [9092, 16176, 16179, 10141, 4795, 16233, 4768, 15892, 4792, 8915])
(1577, [9126, 9475, 15030, 9476, 4514, 15031, 5196, 4512, 5117, 9144])
(1578, [5561, 4669, 7495, 7493, 724, 4357, 4346, 7651, 725, 17870])
(1579, [8857, 8853, 8858, 8854, 5645, 10466, 6292, 7310, 17816, 8179])
(1580, [197, 189, 5133, 194, 192, 4791, 5297, 1034, 5137, 5560])
(1582, [725, 5203, 4771, 5202, 8111, 5206, 5143, 5144, 15119, 4775])
(1585, [2506, 2507, 4691, 5143, 4693, 2290, 16206, 2752, 5166, 5126])
(1586, [5593, 11191, 8635, 11333, 7640, 3962, 5896, 8415, 17547, 8922])
(1588, [520, 4205, 17515, 250, 6043, 8600, 10689, 4605, 11474, 558])
(1589, [7496, 520, 8534, 4977, 9477, 3842, 10227, 15458, 5281, 15030])
(1590, [144

(2878, [8668, 9477, 1943, 8667, 4242, 1212, 15030, 5895, 17835, 8414])
(2879, [7824, 16584, 4018, 207, 9013, 9710, 8884, 17768, 312, 11270])
(2880, [15102, 15101, 15099, 17750, 7662, 7659, 17752, 7661, 12469, 5677])
(2881, [8478, 5293, 6276, 6375, 5463, 17857, 13946, 2154, 12230, 4101])
(2882, [7493, 4634, 4658, 3842, 7494, 13421, 3654, 5739, 16850, 3565])
(2883, [725, 5561, 3805, 5508, 7651, 4220, 7627, 7655, 5044, 7706])
(2884, [8516, 5897, 4713, 8506, 5455, 4726, 4731, 5896, 4792, 4795])
(2885, [3566, 3593, 8526, 8982, 8404, 8668, 3582, 3576, 7592, 7581])
(2887, [5143, 8306, 4634, 4691, 8308, 10068, 4480, 18101, 18094, 8526])
(2888, [4719, 738, 735, 4717, 18330, 4687, 4691, 3541, 7354, 5610])
(2889, [12291, 12328, 12334, 12333, 12332, 12331, 12330, 12329, 12336, 12325])
(2890, [5297, 8481, 10158, 10157, 6793, 5610, 5112, 4024, 10154, 4343])
(2891, [3827, 10316, 10313, 5523, 10914, 5696, 15947, 5291, 9128, 2738])
(2892, [12123, 12066, 3067, 11405, 10219, 10035, 700, 17378, 12332, 123

(4266, [3566, 3582, 4703, 8982, 3595, 3576, 7592, 3594, 8404, 4704])
(4267, [5147, 5142, 6798, 8565, 5128, 4481, 3568, 8638, 5139, 6648])
(4268, [17955, 8638, 5113, 10227, 4657, 197, 8982, 10466, 3922, 4361])
(4269, [9263, 6528, 8414, 8570, 8982, 8569, 9446, 15496, 5645, 8564])
(4270, [3789, 3791, 6405, 5111, 3792, 2853, 8886, 15290, 83, 4394])
(4272, [8473, 6393, 11220, 5112, 5828, 5113, 11218, 5296, 4057, 4977])
(4273, [970, 10949, 1180, 14915, 11038, 1161, 15036, 13465, 3563, 10303])
(4274, [5809, 11250, 12879, 15213, 12334, 12333, 12332, 12331, 12330, 12329])
(4275, [5142, 5396, 6404, 4785, 5402, 6289, 4482, 10227, 1023, 5113])
(4276, [3565, 4657, 5192, 3566, 4658, 5191, 17836, 3593, 4759, 8186])
(4277, [15119, 724, 4658, 4634, 7662, 15114, 10068, 8404, 3593, 7661])
(4278, [75, 76, 6810, 6342, 10889, 11291, 10180, 2981, 16448, 14207])
(4279, [17955, 8638, 5113, 10227, 4657, 197, 8982, 10466, 3922, 4361])
(4280, [8526, 6229, 10227, 7310, 8668, 4280, 3655, 4420, 7493, 8414])
(4281, [

(5436, [7495, 7493, 3654, 5471, 15706, 3842, 7199, 7192, 17748, 16584])
(5437, [18182, 6049, 8724, 6289, 8481, 8635, 6292, 8478, 3827, 3641])
(5438, [6528, 11733, 737, 8008, 11734, 17507, 735, 736, 17506, 738])
(5439, [4284, 9229, 6889, 16224, 7137, 7135, 3822, 17733, 15119, 7127])
(5440, [3566, 3593, 8982, 8404, 3582, 3576, 7592, 126, 8984, 3568])
(5441, [10067, 4747, 5044, 15706, 5568, 7651, 5275, 7536, 7192, 5465])
(5442, [6510, 1041, 7662, 4561, 5322, 5603, 4967, 4944, 18310, 7504])
(5443, [17955, 8638, 5113, 10227, 4657, 197, 8982, 10466, 3922, 4361])
(5444, [5296, 4751, 6880, 5645, 4396, 15892, 50, 187, 4037, 17955])
(5445, [4724, 17836, 4640, 4082, 4671, 4081, 4722, 4721, 17863, 18304])
(5446, [17955, 8638, 5113, 10227, 4657, 197, 8982, 10466, 3922, 4361])
(5447, [5007, 8347, 4629, 7301, 6404, 8186, 6014, 14847, 18286, 8445])
(5448, [6793, 7661, 7662, 7657, 6796, 5603, 10469, 5602, 7174, 6496])
(5449, [4493, 12327, 12326, 12332, 12331, 12330, 12329, 12328, 12334, 12323])
(5450, 

...

(27479, [4844, 3807, 4513, 10232, 6289, 5951, 5510, 5560, 6018, 3815])
(27480, [4703, 4704, 4748, 4629, 5113, 4057, 5044, 6289, 3928, 8481])
(27481, [5732, 13202, 15118, 18332, 8306, 12030, 9084, 4210, 4492, 10003])
(27483, [4703, 4704, 11733, 5192, 18235, 18230, 13837, 18231, 18234, 18238])
(27484, [5193, 3568, 7711, 6530, 5195, 15205, 15204, 5463, 4652, 5661])
(27485, [12410, 7478, 14211, 352, 6453, 16761, 4662, 8970, 873, 16301])
(27486, [4507, 3827, 5523, 10914, 5696, 5291, 6060, 9128, 2818, 2738])
(27487, [8668, 8414, 12469, 2301, 4242, 8526, 10227, 8667, 1212, 4634])
(27488, [3566, 3593, 8982, 8404, 3582, 3576, 7592, 7581, 8984, 3568])
(27490, [16293, 4705, 7662, 7657, 5189, 18143, 5192, 4712, 5191, 5714])
(27491, [6778, 4689, 4690, 5678, 4688, 5677, 6900, 6038, 4486, 4701])
(27492, [18158, 7375, 18157, 7374, 18156, 16161, 18166, 15951, 18143, 2162])
(27493, [4713, 5189, 4726, 5192, 4727, 5191, 6657, 4711, 4722, 4658])
(27494, [4124, 4123, 4193, 2775, 4034, 9229, 4630, 7326, 1618

(28508, [17955, 8638, 5113, 10227, 4657, 197, 8982, 10466, 3922, 4361])
(28509, [189, 192, 187, 194, 4738, 16186, 6282, 17560, 7699, 16182])
(28511, [5297, 4696, 4687, 4693, 4695, 3541, 7583, 4703, 5926, 5294])
(28512, [16176, 16179, 4967, 4359, 4059, 16200, 5637, 4069, 4352, 11214])
(28513, [10112, 8914, 8915, 10846, 9091, 10912, 14887, 4483, 7243, 10245])
(28514, [9092, 189, 192, 3252, 187, 194, 5650, 232, 8915, 8914])
(28515, [7496, 8515, 10067, 7493, 9476, 12257, 15030, 8508, 4035, 5895])
(28516, [17955, 8638, 5113, 10227, 4657, 197, 8982, 10466, 3922, 4361])
(28517, [16186, 17559, 7703, 4187, 657, 16226, 17560, 16181, 16578, 17719])
(28518, [4728, 15102, 15101, 15099, 6514, 4638, 4712, 4637, 1943, 12469])
(28519, [4657, 4658, 9302, 9224, 4034, 9305, 10813, 13837, 4362, 9104])
(28520, [16186, 4191, 4190, 4187, 467, 2776, 657, 17560, 17558, 16226])
(28521, [7368, 6201, 6202, 6203, 4279, 3642, 7687, 6205, 15728, 8202])
(28522, [17955, 8638, 5113, 10227, 4657, 197, 8982, 10466, 3922, 

(29384, [12197, 12196, 12198, 14639, 5973, 12330, 12334, 12333, 12332, 12331])
(29385, [8031, 7703, 4362, 4949, 9224, 4955, 4000, 4365, 16182, 4123])
(29387, [17730, 14216, 4102, 14192, 6497, 17541, 16596, 5266, 5235, 4488])
(29388, [9302, 9305, 7584, 6136, 7602, 17661, 3834, 4693, 3643, 16426])
(29389, [5606, 17750, 8186, 17752, 5921, 6018, 4355, 10466, 5918, 2834])
(29390, [4723, 4722, 10227, 4721, 3806, 6393, 5175, 5176, 5246, 5248])
(29391, [17750, 3569, 7659, 7662, 17752, 6530, 7657, 6136, 5602, 3790])
(29392, [521, 8535, 4713, 9709, 4731, 8565, 4983, 4711, 4727, 4985])
(29393, [8668, 739, 15907, 738, 735, 8445, 4751, 15908, 736, 7606])
(29394, [8223, 10347, 4253, 4252, 6521, 5339, 10222, 9153, 16349, 9968])
(29395, [5202, 5203, 5206, 10585, 6392, 10081, 6390, 44, 11785, 10587])
(29396, [6167, 7494, 9035, 11933, 10914, 4439, 7192, 5731, 8261, 193])
(29397, [4528, 6375, 5940, 9014, 4687, 5934, 10068, 4696, 5926, 1944])
(29399, [4657, 7885, 4702, 4658, 7884, 4704, 8506, 8445, 7888, 

(30059, [4689, 5933, 3514, 5261, 4719, 3518, 4701, 4720, 3519, 4660])
(30061, [3626, 7687, 3627, 14251, 16700, 6785, 16706, 3624, 16698, 4452])
(30062, [8309, 5127, 4396, 8203, 14353, 4806, 6280, 9910, 8012, 7600])
(30064, [17955, 8638, 5113, 10227, 4657, 197, 8982, 10466, 3922, 4361])
(30065, [2536, 14060, 16434, 15767, 1221, 12334, 12332, 12328, 12331, 12333])
(30066, [7198, 7197, 4970, 8922, 4968, 9932, 5838, 4988, 6331, 14727])
(30067, [7259, 5007, 5006, 5401, 530, 532, 9263, 6280, 5942, 5720])
(30068, [6526, 3592, 9092, 8414, 4827, 9093, 6528, 3873, 8914, 3597])
(30069, [4124, 4123, 7661, 7662, 7657, 4183, 18276, 4034, 16306, 17732])
(30070, [17955, 8638, 5113, 10227, 4657, 197, 8982, 10466, 3922, 4361])
(30071, [6793, 4719, 4720, 5143, 5144, 6796, 4785, 4762, 4487, 5135])
(30072, [17955, 8638, 5113, 10227, 4657, 197, 8982, 10466, 3922, 4361])
(30073, [4128, 5624, 6731, 4214, 5949, 5677, 5681, 7687, 18343, 5942])
(30074, [10067, 4714, 3806, 15706, 5275, 4629, 7192, 5220, 17748, 47

(30726, [18235, 18230, 18233, 18158, 18231, 18238, 18236, 18148, 18168, 4719])
(30727, [4657, 4634, 4711, 4712, 4727, 4731, 4728, 4726, 390, 6881])
(30728, [189, 192, 194, 14800, 6807, 5297, 10067, 8638, 5560, 126])
(30729, [5528, 4212, 4546, 5547, 8899, 18051, 10387, 12336, 12334, 12335])
(30730, [17953, 15892, 17952, 189, 7661, 5610, 6780, 4057, 7642, 5114])
(30731, [189, 197, 10167, 127, 194, 192, 17952, 8179, 3387, 14800])
(30732, [8508, 8515, 8516, 8502, 8518, 17637, 9111, 750, 10741, 4328])
(30733, [3509, 5285, 8702, 10575, 4531, 8459, 4235, 4739, 3079, 7291])
(30735, [189, 192, 194, 8445, 8414, 15101, 15099, 15100, 8666, 4210])
(30736, [6526, 4785, 1023, 8008, 5144, 5133, 4784, 5118, 4482, 5119])
(30737, [17955, 8638, 5113, 10227, 4657, 197, 8982, 10466, 3922, 4361])
(30738, [7626, 10178, 8296, 3427, 2545, 10874, 3148, 12331, 12334, 12328])
(30739, [17955, 8638, 5113, 10227, 4657, 197, 8982, 10466, 3922, 4361])
(30740, [15123, 15156, 15129, 16280, 6454, 3820, 819, 11101, 14116, 